In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db, auth, firestore
import pandas as pd
from datetime import datetime

In [2]:

#Firebase database 인증 및 앱 초기화
cred = credentials.Certificate('findsea-77ce8-firebase-adminsdk-dkvm6-74ecc418c0.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()


In [3]:
data = pd.read_csv('./데이터/임시.csv')

data = data.drop(['품종'],axis=1)
data.columns = ['date','catch(kg)','ws(mPs)','atm(hPa)','humid(%)','tmp(C)','wTmp(C)','waveH(m)','waveDH(m)','wP(sec)','wti']
data['date'] = pd.to_datetime(data['date'])
data['date']=data['date'].dt.strftime('%Y%m%d')
data['date']=data['date'].astype(int)

In [14]:
for year in range(2013, 2024):
  year_data = data['date'] <= ((year * 10000) + 1231)
  for month in range(1, 13) :
    month_data = data['date'] <= ((((year * 10000) + (month * 100)) + 31))
    for day in range(1, 32):
      day_data = data['date'] == (((year*10000) + (month * 100) + day))
      day_data_df = data[day_data]
      if day_data_df.empty==True:
        continue
      # day_data_df['createdTime'] = datetime.timestamp()
      if month < 10 :
        doc_ref = db.collection(u'fishes').document(u'scombers').collection(u'raw').  document(u'south').collection(str(year)).document('0'+str((month*100)+day).zfill(2))
      else: 
        doc_ref = db.collection(u'fishes').document(u'scombers').collection(u'raw').document(u'south').collection(str(year)).document(str((month*100)+day).zfill(2))

      doc_ref.set({
          u'date': day_data_df['date'].tolist()[0],
          u'catch(kg)': day_data_df['catch(kg)'].tolist()[0],
          u'wti': day_data_df[u'wti'].tolist()[0],
          u'ws(mPs)': day_data_df[u'ws(mPs)'].tolist()[0],
          u'atm(hPa)': day_data_df[u'atm(hPa)'].tolist()[0],
          u'humid(%)': day_data_df[u'humid(%)'].tolist()[0],
          u'tmp(C)': day_data_df[u'tmp(C)'].tolist()[0],
          u'wTmp(C)': day_data_df[u'wTmp(C)'].tolist()[0],
          u'waveH(m)': day_data_df[u'waveH(m)'].tolist()[0],
          u'waveDH(m)': day_data_df[u'waveDH(m)'].tolist()[0],
          u'wP(sec)': day_data_df[u'wP(sec)'].tolist()[0],
          u'createdTime': firestore.SERVER_TIMESTAMP
    })